# Usando os truques de treinamento de redes neurais!

A documentação do Keras está em https://www.tensorflow.org/api_docs/python/tf/keras/

In [1]:
%matplotlib inline
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

In [2]:
import tensorflow as tf
from tensorflow import keras

# Lê o dataset CIFAR-10, https://keras.io/api/datasets/cifar10/
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 75s 0us/step


Fonte: https://www.cs.toronto.edu/~kriz/cifar.html

<img src='cifar.png'></img>

O conjunto de treinamento tem $50000$ imagens de $32 \times 32$ pixels, com $3$ canais de cor. Portanto, o *array* `X_train_full` contendo esses dados tem formato $(\text{amostras}, \text{linhas}, \text{colunas}, \text{canais}) = (50000, 32, 32, 3)$

In [3]:
print(X_train_full.shape)

(50000, 32, 32, 3)


Cada canal de cor é representado como um inteiro entre $0$ e $255$, e isso cabe em um único *byte*. Para economizar espaço, o tipo de dados do *array* então será `uint8`, significando "inteiro sem-sinal de 8 bits". Ou para o povo do C/C++, o famoso `unsigned char`.

In [4]:
print(X_train_full.dtype)

uint8


A variável dependente `y_train_full` tem $50000$ amostras e apenas um valor por amostra. Para se manter consistente com os formatos de dados do *framework* Tensorflow, a variável dependente será representada por uma matriz-coluna de tamanho $(50000, 1)$ ao invés de um array simples de tamanho $(50000,)$

In [5]:
print(y_train_full.shape)

(50000, 1)


Como as classes são inteiros entre $0$ e $9$, um `uint8` basta para armazená-las

In [6]:
print(y_train_full.dtype)

uint8


Já o conjunto de testes tem $10000$ amostras, observe os tamanhos e tipos de dado de `X_test` e `y_test`:

In [7]:
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

(10000, 32, 32, 3) uint8
(10000, 1) uint8


O conjunto de teste, como de costume, é inviolável. Vamos dividir o conjunto de treinamento mais uma vez em duas partes: um conjunto que realmente será usado para treinamento, e outro para validação (que é o conjunto de testes de dentro do conjunto de treinamento pleno, e então não é inviolável).

Vamos separar $5000$ amostras para validação.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=5000
)

print("X_train", X_train.shape, X_train.dtype)
print("y_train", y_train.shape, y_train.dtype)
print("X_valid", X_valid.shape, X_valid.dtype)
print("y_valid", y_valid.shape, y_valid.dtype)

X_train (45000, 32, 32, 3) uint8
y_train (45000, 1) uint8
X_valid (5000, 32, 32, 3) uint8
y_valid (5000, 1) uint8


Antes de mais nada, vamos criar um *baseline* de desempenho para comparação: a Random Forest.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
%%time
model = RandomForestClassifier(n_jobs=-1, random_state=42)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

Wall time: 1min 36s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [11]:
y_pred = model.predict(X_valid.reshape(X_valid.shape[0], -1))
print(accuracy_score(y_valid.ravel(), y_pred))

0.4646


Ok, esse é o *score* a ser vencido!

Vamos agora construir uma rede neural das antigas: nada profunda, e com função de ativação sigmoide. Vamos usar um `batch_size` grande, pois o dataset é pequeno, não precisamos de batches muito pequenos (o tamanho padrão é 32 no TensorFlow). Veremos o desempenho:

In [12]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "inicial")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 2s 43ms/step - loss: 2.2663 - accuracy: 0.1528 - val_loss: 2.2071 - val_accuracy: 0.1936
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 2.1921 - accuracy: 0.2000 - val_loss: 2.1658 - val_accuracy: 0.2278
Epoch 3/100
44/44 [==============================] - 1s 14ms/step - loss: 2.1484 - accuracy: 0.2297 - val_loss: 2.1303 - val_accuracy: 0.2442
Epoch 4/100
44/44 [==============================] - 1s 16ms/step - loss: 2.1118 - accuracy: 0.2501 - val_loss: 2.0930 - val_accuracy: 0.2534
Epoch 5/100
44/44 [==============================] - 1s 16ms/step - loss: 2.0851 - accuracy: 0.2625 - val_loss: 2.0710 - val_accuracy: 0.2682
Epoch 6/100
44/44 [==============================] - 1s 13ms/step - loss: 2.0682 - accuracy: 0.2636 - val_loss: 2.0490 - val_accuracy: 0.2720
Epoch 7/100
44/44 [==============================] - 1s 15ms/step - loss: 2.0381 - accuracy: 0.2809 - val_loss: 2.0269 - val_accuracy: 0.2924
Epoch 

[1.7710466384887695, 0.3675999939441681]

Vamos partir para uma rede profunda agora. Vamos criar $10$ camadas *fully-connected*, função de ativação "elu", inicialização de pesos adequada para a respectiva função de ativação, e otimizador "adam". Vamos ver se melhora:

In [13]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "profunda")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 3s 57ms/step - loss: 70.5708 - accuracy: 0.1037 - val_loss: 10.0872 - val_accuracy: 0.1050
Epoch 2/100
44/44 [==============================] - 1s 19ms/step - loss: 4.2841 - accuracy: 0.1338 - val_loss: 2.7999 - val_accuracy: 0.1730
Epoch 3/100
44/44 [==============================] - 1s 23ms/step - loss: 2.5891 - accuracy: 0.1841 - val_loss: 2.5723 - val_accuracy: 0.1878
Epoch 4/100
44/44 [==============================] - 1s 21ms/step - loss: 2.7397 - accuracy: 0.1813 - val_loss: 2.6224 - val_accuracy: 0.1814
Epoch 5/100
44/44 [==============================] - 1s 18ms/step - loss: 2.4065 - accuracy: 0.2098 - val_loss: 2.1766 - val_accuracy: 0.2232
Epoch 6/100
44/44 [==============================] - 1s 19ms/step - loss: 2.1609 - accuracy: 0.2338 - val_loss: 2.2337 - val_accuracy: 0.2154
Epoch 7/100
44/44 [==============================] - 3s 70ms/step - loss: 2.1182 - accuracy: 0.2445 - val_loss: 2.1951 - val_accuracy: 0.2638
Epoc

[1.5755152702331543, 0.44179999828338623]

Muito melhor! Agora acertamos em torno de $43\%$ das classes!

A técnica de "batch normalization" pode melhorar ainda mais a convergência do modelo, vamos experimentar:

In [14]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "batchnorm")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 5s 106ms/step - loss: 1.8688 - accuracy: 0.3441 - val_loss: 4.2052 - val_accuracy: 0.1598
Epoch 2/100
44/44 [==============================] - 2s 35ms/step - loss: 1.5413 - accuracy: 0.4480 - val_loss: 2.4820 - val_accuracy: 0.2864
Epoch 3/100
44/44 [==============================] - 1s 33ms/step - loss: 1.4262 - accuracy: 0.4916 - val_loss: 1.8602 - val_accuracy: 0.3926
Epoch 4/100
44/44 [==============================] - 2s 36ms/step - loss: 1.3471 - accuracy: 0.5195 - val_loss: 1.6093 - val_accuracy: 0.4454
Epoch 5/100
44/44 [==============================] - 1s 34ms/step - loss: 1.2766 - accuracy: 0.5444 - val_loss: 1.5343 - val_accuracy: 0.4664
Epoch 6/100
44/44 [==============================] - 1s 34ms/step - loss: 1.2152 - accuracy: 0.5662 - val_loss: 1.4994 - val_accuracy: 0.4830
Epoch 7/100
44/44 [==============================] - 2s 35ms/step - loss: 1.1645 - accuracy: 0.5832 - val_loss: 1.4869 - val_accuracy: 0.4864
Epoch

[4.832778453826904, 0.47200000286102295]

Uau. Isso sim que é *overfitting*! Para resolver o problema do *overfitting* vamos adotar o EarlyStopping:

In [15]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "earlystopping")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 6s 126ms/step - loss: 1.8688 - accuracy: 0.3441 - val_loss: 4.2052 - val_accuracy: 0.1598
Epoch 2/100
44/44 [==============================] - 2s 38ms/step - loss: 1.5413 - accuracy: 0.4480 - val_loss: 2.4820 - val_accuracy: 0.2864
Epoch 3/100
44/44 [==============================] - 2s 37ms/step - loss: 1.4262 - accuracy: 0.4916 - val_loss: 1.8602 - val_accuracy: 0.3926
Epoch 4/100
44/44 [==============================] - 2s 38ms/step - loss: 1.3471 - accuracy: 0.5195 - val_loss: 1.6093 - val_accuracy: 0.4454
Epoch 5/100
44/44 [==============================] - 2s 35ms/step - loss: 1.2766 - accuracy: 0.5444 - val_loss: 1.5343 - val_accuracy: 0.4664
Epoch 6/100
44/44 [==============================] - 2s 36ms/step - loss: 1.2152 - accuracy: 0.5662 - val_loss: 1.4994 - val_accuracy: 0.4830
Epoch 7/100
44/44 [==============================] - 2s 36ms/step - loss: 1.1645 - accuracy: 0.5832 - val_loss: 1.4869 - val_accuracy: 0.4864
Epoch

[1.7146939039230347, 0.49639999866485596]

Melhorou mais! Mas ainda temos o problema do *overfitting*, vamos tentar a técnica do "drop-out" para ver se melhora:

In [16]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "dropout")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 4s 99ms/step - loss: 2.2026 - accuracy: 0.2366 - val_loss: 4.9245 - val_accuracy: 0.1416
Epoch 2/100
44/44 [==============================] - 1s 20ms/step - loss: 1.8559 - accuracy: 0.3237 - val_loss: 3.0027 - val_accuracy: 0.2588
Epoch 3/100
44/44 [==============================] - 1s 20ms/step - loss: 1.7512 - accuracy: 0.3692 - val_loss: 2.0210 - val_accuracy: 0.3646
Epoch 4/100
44/44 [==============================] - 1s 20ms/step - loss: 1.6866 - accuracy: 0.3951 - val_loss: 1.7284 - val_accuracy: 0.4208
Epoch 5/100
44/44 [==============================] - 1s 20ms/step - loss: 1.6377 - accuracy: 0.4138 - val_loss: 1.6215 - val_accuracy: 0.4388
Epoch 6/100
44/44 [==============================] - 1s 20ms/step - loss: 1.5986 - accuracy: 0.4285 - val_loss: 1.5641 - val_accuracy: 0.4470
Epoch 7/100
44/44 [==============================] - 1s 20ms/step - loss: 1.5641 - accuracy: 0.4400 - val_loss: 1.5246 - val_accuracy: 0.4634
Epoch 

[1.3109263181686401, 0.5460000038146973]

Melhorou ainda mais! E agora não tem mais tanto overfitting! Já que é assim, vamos aumentar ainda mais a rede e ver se melhora: vamos colocar $20$ camadas!

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "mais_profunda")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 10s 233ms/step - loss: 2.4776 - accuracy: 0.1452 - val_loss: 2.9542 - val_accuracy: 0.1252
Epoch 2/100
44/44 [==============================] - 2s 41ms/step - loss: 2.0676 - accuracy: 0.2350 - val_loss: 2.9201 - val_accuracy: 0.2150
Epoch 3/100
44/44 [==============================] - 2s 41ms/step - loss: 1.9330 - accuracy: 0.2919 - val_loss: 2.4435 - val_accuracy: 0.3004
Epoch 4/100
44/44 [==============================] - 2s 38ms/step - loss: 1.8586 - accuracy: 0.3195 - val_loss: 2.0578 - val_accuracy: 0.3506
Epoch 5/100
44/44 [==============================] - 2s 37ms/step - loss: 1.8067 - accuracy: 0.3387 - val_loss: 1.8766 - val_accuracy: 0.3774
Epoch 6/100
44/44 [==============================] - 1s 34ms/step - loss: 1.7660 - accuracy: 0.3584 - val_loss: 1.8114 - val_accuracy: 0.3828
Epoch 7/100
44/44 [==============================] - 2s 39ms/step - loss: 1.7297 - accuracy: 0.3725 - val_loss: 1.7171 - val_accuracy: 0.4098
Epoc

[1.3490664958953857, 0.5271999835968018]

Parece que não melhorou muito, estava bom do jeito anterior. Para terminar, vamos usar um escalonador de taxa de aprendizado para melhorar a velocidade de convergência do nosso modelo.

In [18]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "escalonador")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        )
    )
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    )
)

s = 90 * len(X_train) // 1024  # number of steps in 20 epochs (batch size = 1024)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 5s 102ms/step - loss: 2.0087 - accuracy: 0.2845 - val_loss: 49.1815 - val_accuracy: 0.1342
Epoch 2/100
44/44 [==============================] - 1s 18ms/step - loss: 1.7644 - accuracy: 0.3612 - val_loss: 4.6598 - val_accuracy: 0.3184
Epoch 3/100
44/44 [==============================] - 1s 21ms/step - loss: 1.6752 - accuracy: 0.3996 - val_loss: 2.3738 - val_accuracy: 0.3702
Epoch 4/100
44/44 [==============================] - 1s 21ms/step - loss: 1.6065 - accuracy: 0.4268 - val_loss: 1.9051 - val_accuracy: 0.4002
Epoch 5/100
44/44 [==============================] - 1s 21ms/step - loss: 1.5568 - accuracy: 0.4451 - val_loss: 1.6218 - val_accuracy: 0.4428
Epoch 6/100
44/44 [==============================] - 1s 20ms/step - loss: 1.5216 - accuracy: 0.4562 - val_loss: 1.6238 - val_accuracy: 0.4408
Epoch 7/100
44/44 [==============================] - 1s 20ms/step - loss: 1.4756 - accuracy: 0.4702 - val_loss: 1.5096 - val_accuracy: 0.4748
Epoc

[1.332063913345337, 0.5450000166893005]

In [19]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.2917 - accuracy: 0.5485


[1.291690468788147, 0.5485000014305115]

**Atividade**: Refaça a atividade da aula anterior (Fashion MNIST) mas tente usar os truques de melhoramento de desempenho das redes neurais vistos aqui nesta aula.